In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from datetime import time
from datetime import datetime
import numpy as np
import pandas as pd
from deap_r import base, creator, tools, gp, algorithms
#from operators import *
import operators_v4
from fitness import *
#import pyarrow.feather as feather
import gc
import argparse
import random
import json
import warnings
import pathlib
import pickle
import tqdm
warnings.filterwarnings('ignore')
#import matplotlib.pyplot as plt
import sys
import Tool
import fitness
import GAStockSelection_v4
import sys
sys.path.append("C:/Users/User/我的雲端硬碟 (owen.lin@mutual-boost.com)/MBQ_Tej_v2/MBQ_tej_v2")
import MBQ_tej_v2_manager
import config
import Tool
import operators_v4

In [3]:
# new_expr_dict = {
#     # # 成交量/價新變化
#     # 'vol_ratio_5_20': 'Volume_1000_Shares.rolling(5).mean() / Volume_1000_Shares.rolling(20).mean()',
#     # 'vwap_diff': 'Vwap - Adj_Close',
#     # 'price_range_ratio': '(Adj_High - Adj_Low) / Adj_Low',
#     # 'intraday_range': '(Adj_High - Adj_Low) / Adj_Open',
    
#     # # 當沖與總量變化
#     # 'DayTrade_to_Turnover': 'Margin_Day_Trading_Vol / Turnover',
#     # 'DayTrade_value_ratio': 'Margin_Day_Trading_Amt / Value_Dollars',
    
#     # # 統計動量類（非動能）
#     # 'rolling_skew_20': 'daily_ret.rolling(20).skew()',
#     # 'rolling_kurt_20': 'daily_ret.rolling(20).kurt()',
    
#     # # 創新性日內表達式
#     # 'OpenClose_diff': 'Adj_Open - Adj_Close',
#     # 'HighOpen_diff': 'Adj_High - Adj_Open',
#     # 'LowOpen_diff': 'Adj_Open - Adj_Low',
#     # 'Body_size': 'abs(Adj_Close - Adj_Open)',
#     # 'Upper_shadow': 'Adj_High - Adj_Close.where(Adj_Close > Adj_Open, Adj_Open)',
#     # 'Lower_shadow': 'Adj_Low - Adj_Open.where(Adj_Open > Adj_Close, Adj_Close)',
    
#     # # 價量相關因子
#     # 'Volume_ret_corr_20': 'Adj_Close.pct_change().rolling(20).corr(Volume_1000_Shares)',
#     # 'Volume_VIX_like': 'Volume_1000_Shares.pct_change().rolling(20).std()',
    
#      # 成交量分析類
#     'UpVol': '(Adj_Close > Adj_Close.shift()) * Volume',
#     'DownVol': '(Adj_Close < Adj_Close.shift()) * Volume',
#     'UpDownVolRatio20': 'UpVol.rolling(20).sum() / DownVol.rolling(20).sum()',

#     # 價格與均線的乖離
#     'PriceAboveMA20': 'Adj_Close / Adj_Close.rolling(20).mean() - 1',

#     # 波動與成交量的比例
#     'PriceVolatilityRatio': '((Adj_High - Adj_Low) / Adj_Close).rolling(5).mean() / Turnover.rolling(5).mean()',

#     # 盤中波動幅度
#     'HighLowRangeNorm': '(Adj_High - Adj_Low) / Adj_Close.rolling(5).mean()',

#     # 量變
#     'AccumVolChange5': 'Volume.diff().rolling(5).sum() / Volume.shift(5)',

#     # 區間突破強度
#     'BreakoutStrength20': '(Adj_Close - Adj_Close.rolling(20).min()) / (Adj_Close.rolling(20).max() - Adj_Close.rolling(20).min())',

#     # 收盤價與高低點距離
#     'CloseToHighRatio': '(Adj_High - Adj_Close) / (Adj_High - Adj_Low + 1e-6)',
#     'CloseToLowRatio': '(Adj_Close - Adj_Low) / (Adj_High - Adj_Low + 1e-6)',

#     # 價量相關性
#     'TurnVolCorr20': 'Adj_Close.pct_change().rolling(20).corr(Volume)',

#     # 實體 K 線長度平均
#     'BodyStrength20': '(Adj_Close - Adj_Open).abs().rolling(20).mean() / Adj_Close.rolling(20).mean()',
    
#     'RealizedVol20': '(Adj_Close.pct_change()**2).rolling(20).sum()**0.5',
#     'IntradayVolatility': '(Adj_High - Adj_Low) / Adj_Close.shift()',
#     'PriceStdTrend20': 'Adj_Close.rolling(20).std() / Adj_Close.rolling(60).std()',
#     'RangeRatio': '(Adj_High - Adj_Low) / (Adj_Close.rolling(20).max() - Adj_Close.rolling(20).min())',
#     'VolPriceDeviation': 'Adj_Close - Adj_Close.rolling(20).mean() + Volume - Volume.rolling(20).mean()',
#     'TurnoverStdRatio': 'Turnover.rolling(5).std() / Turnover.rolling(20).std()',
#     'OpenCloseSpread': 'Adj_Open / Adj_Close - 1',
#     'CloseHighLowMeanDiff': 'Adj_Close - ((Adj_High + Adj_Low) / 2)',
#     'ClosenessToHigh': '(Adj_Close - Adj_Low) / (Adj_High - Adj_Low)',
#     'GapReturn': 'Adj_Open / Adj_Close.shift(1) - 1',
#     'VolLeadPrice': 'Volume.shift(1) * (Adj_Close / Adj_Close.shift(1) - 1)',
#     'PriceLeadVol': '(Adj_Close / Adj_Close.shift(1) - 1).shift(1) * Volume',
#     'TurnoverLeadRet': 'Turnover.shift(1) * (Adj_Close / Adj_Close.shift(1) - 1)',
# }


In [4]:
def creat_pop_from_str_list(pop_fun_str_list:list,pset)->list:
    return [creator.Individual.from_string(ind_fun_str, pset) for ind_fun_str in pop_fun_str_list]#直接定義因子式
def 剔除高度相关(df_result,ts_result,Corr_threshold:float = 0.95):
    要删除的清单 = list()
    #根據因子本身序列評價單因子績效並排序
    #cv_df = ts_result.mean() / ts_result.std()
    #ts_result = ts_result[cv_df.sort_values(ascending = False).index]
    ts_result = ts_result[df_result.sort_values('fitness',ascending=False)['cal'].to_list()]
    Corr_df = ts_result.corr().abs()
    for factor_expr in ts_result.columns:
        if factor_expr not in 要删除的清单:
            Corr_ = Corr_df.loc[factor_expr:,factor_expr].iloc[1:]
            要删除的清单+=Corr_[Corr_>Corr_threshold].index.to_list()
    要删除的清单 = list(set(要删除的清单))
    if 要删除的清单:
        print(f'发现:{len(要删除的清单)}个表达式与其他其他表达式相关超过{Corr_threshold}，执行删除！！！')
    else:
        print(f'没有相关超过{Corr_threshold}的表达式！！！')
    df_result = df_result.loc[~df_result['cal'].isin(要删除的清单)]
    ts_result = ts_result[df_result['cal'].to_list()]
    return df_result,ts_result

In [5]:
start_time = datetime.now()
print(f"Init variables:{start_time}.")
args = json.load(open("./config.json", encoding="utf-8"))
if args['OUTPUT_DATA_PATH'][:2] == './':
    args['OUTPUT_DATA_PATH'] = str(pathlib.Path().absolute())+args['OUTPUT_DATA_PATH'][1:]
# Data preprocessing
Handler_cache_path = args['Handler_cache_path']
print(f"Load Data : {Handler_cache_path}")
CacheHandler = Tool.Handler(path = Handler_cache_path,data_type = args['data_type'])

Con = eval(args['Con'],CacheHandler)
target = eval(args['TARGET_Expr'],CacheHandler)
target = target[np.isfinite(target) & Con]
print(f"Target : ({args['TARGET_Expr']})[{args['Con']}]")

Init variables:2025-04-08 09:28:53.996739.
Load Data : C:\Users\User\Documents\MBQ_TW_DB
Target : (Adj_close.shift(-21) / Adj_close.shift(-1) - 1)[(Value_Dollars.rolling(5).mean() > 0)]


In [6]:
使用太多次需剔除的data_name = []
ts_result_path = os.path.join(
    os.path.dirname(args['OUTPUT_DATA_PATH']),
    args['OUTPUT_DATA_PATH'].split('/')[-1].replace('.xlsx', '') + '.parquet'
)

if os.path.exists(args['OUTPUT_DATA_PATH']) and os.path.exists(ts_result_path):
    ts_result = pd.read_parquet(ts_result_path)
    df_result = pd.read_excel(args['OUTPUT_DATA_PATH'], index_col=0)

    # 清理
    df_result, ts_result = 剔除高度相关(df_result, ts_result, args['Corr_threshold'])
    df_result['use_data'] = df_result['cal'].apply(lambda X: fitness.parse_expression(X, False)[1])
    df_result = df_result.loc[df_result['fitness'] > args['FIT_THRES']]

    print(f"{args['OUTPUT_DATA_PATH']} 已存在，載入歷史 {len(df_result)} 個歷史因子")
    use_data_df = pd.DataFrame([
        (k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values
    ], columns=['key', 'value'])

    use_factor = use_data_df.groupby('value')['key'].apply(list)
    use_factor_count = use_factor.str.len().sort_values(ascending=False)
    使用太多次需剔除的data_name = use_factor_count.loc[
        use_factor_count >= args['use_data_count_threshold']
    ].index.to_list()
    if 使用太多次需剔除的data_name:
        print(f'剔除: {使用太多次需剔除的data_name}')

    print(f"Mega Factor Sharpe Up to: {ts_result.mean(axis=1).mean() / ts_result.mean(axis=1).std() * 252**0.5}")
    print(f"Mega Factor Cagr Up to: {Tool.CAGR(ts_result.mean(axis=1))}")
else:
    print("尚未有舊資料，初始化空的 ts_result 與 df_result")
    df_result = pd.DataFrame(columns=['cal', 'fitness', 'use_data'])
    ts_result = pd.DataFrame()
count = 0
if args['AMT']=="None" or args['AMT']=="inf": args['AMT'] = np.inf

没有相关超过0.8的表达式！！！
c:\Users\User\Desktop\pc\stcok/Results/Results.xlsx 已存在，載入歷史 20 個歷史因子
Mega Factor Sharpe Up to: 12.29465413203126
Mega Factor Cagr Up to: 91.99453956861807


In [ ]:
while len(df_result) < args['AMT']:
    cash_list = CacheHandler.cash_list()
    require_cols = [_ for _ in cash_list if '_mrq_0' in _] + [_ for _ in cash_list if '_mrq_' not in _]
    require_cols = list(set(require_cols) - set(使用太多次需剔除的data_name) - set(['Alpha102']) - set( ['atten_fg','disp_fg','sbadt_fg']))
    len(require_cols)
    random.seed(count+datetime.timestamp(start_time))#使用當下時間建立隨機種子
    print(f"Initialize Algorithm Use : {len(require_cols)} cols")
    time_constraint_from = args['DATE_FROM']
    time_constraint_to = args['DATE_TO']

    df_type = pd.core.frame.DataFrame
    pset = gp.PrimitiveSetTyped(
        "MAIN", [df_type] * len(require_cols), df_type, require_cols)
    for Fun_name,Fun_object in operators_v4.Alpha_F.items():
        Fun_object.__name__ = Fun_name
        locals()[Fun_name] = Fun_object
    # normal operator
    pset.addPrimitive(abs, [df_type], df_type)
    pset.addPrimitive(log, [df_type], df_type)
    pset.addPrimitive(sign, [df_type], df_type)
    pset.addPrimitive(cs_rank, [df_type], df_type)
    pset.addPrimitive(delay, [df_type, int], df_type)
    pset.addPrimitive(correlation, [df_type, df_type, int], df_type)
    pset.addPrimitive(covariance, [df_type, df_type, int], df_type)
    pset.addPrimitive(cs_scale, [df_type, int], df_type)
    pset.addPrimitive(ts_delta, [df_type, int], df_type)
    pset.addPrimitive(signedpower, [df_type, int], df_type)
    pset.addPrimitive(ts_decay_linear, [df_type, int], df_type)
    pset.addPrimitive(ts_min, [df_type, int], df_type)
    pset.addPrimitive(ts_max, [df_type, int], df_type)
    pset.addPrimitive(ts_argmin, [df_type, int], df_type)
    pset.addPrimitive(ts_argmax, [df_type, int], df_type)
    pset.addPrimitive(ts_rank, [df_type, int], df_type)
    pset.addPrimitive(min, [df_type, df_type], df_type)
    pset.addPrimitive(max, [df_type, df_type], df_type)
    pset.addPrimitive(ts_sum, [df_type, int], df_type)
    pset.addPrimitive(ts_product, [df_type, int], df_type)
    pset.addPrimitive(ts_stddev, [df_type, int], df_type)
    #pset.addPrimitive(where, [df_type, df_type,df_type], df_type)
    pset.addPrimitive(ts_mean, [df_type, int], df_type)
    
    pset.addPrimitive(add, [df_type, df_type], df_type)
    pset.addPrimitive(sub, [df_type, df_type], df_type)
    pset.addPrimitive(mul, [df_type, df_type], df_type)
    pset.addPrimitive(truediv, [df_type, df_type], df_type)
    pset.addPrimitive(Zscore, [df_type], df_type)
    pset.addPrimitive(非负化处理, [df_type], df_type)
    pset.addPrimitive(Neutralization, [df_type, df_type], df_type)
    pset.addPrimitive(Factor_to_weight, [df_type], df_type)
    #pset.addPrimitive(mean, [df_type, df_type], df_type)
    #pset.addPrimitive(std, [df_type, df_type], df_type)
    #def SUE(df_type):pass
    #class fin_int(int):pass
    #pset.addPrimitive(SUE, [df_type, fin_int], df_type)
    
    # === 加入 Terminal (int) ===
    #pset.addTerminal(4, fin_int)
    #pset.addTerminal(8, fin_int)
    #pset.addTerminal(12, fin_int)

    pset.addTerminal(1, int)
    pset.addTerminal(2, int)
    pset.addTerminal(3, int)
    pset.addTerminal(5, int)
    pset.addTerminal(10, int)
    pset.addTerminal(15, int)
    pset.addTerminal(20, int)
    pset.addTerminal(30, int)
    pset.addTerminal(60, int)
    pset.addTerminal(120, int)
    pset.addTerminal(250, int)
    
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", gp.PrimitiveTree,
                    fitness=creator.FitnessMax)

    scope = {k: v for k, v in locals().items() if k in require_cols}
    scope['ts_result'] = ts_result

    if not ts_result.empty:
        scope['mega_score'] = Tool.CAGR(ts_result.mean(axis=1))
    else:
        scope['mega_score'] = 0
    toolbox = base.Toolbox()
    #int_terminal_types = [int,fin_int]
    int_terminal_types = [int]
    toolbox.register("expr", gp.genHalfAndHalf, pset=pset,
                        min_=0, max_=3, terminal_types=int_terminal_types)
    toolbox.register("individual", tools.initIterate,
                        creator.Individual, toolbox.expr)
    toolbox.register("population", tools.initRepeat,
                        list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)
    toolbox.register("evaluate", eval(args['FIT_FUNC']), toolbox_compile=toolbox.compile,
                        target=target, time_constraint_from=time_constraint_from, time_constraint_to=time_constraint_to, scope=scope, require_cols=require_cols)
    toolbox.register("select", tools.selTournament, tournsize=10)
    toolbox.register("mate", gp.cxOnePoint)
    toolbox.register("expr_mut", gp.genFull, min_=0, max_=1,
                        terminal_types=int_terminal_types)
    toolbox.register("mutate", gp.mutUniform,
                        expr=toolbox.expr_mut, pset=pset)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.nanmean)
    stats.register("std", np.nanstd)
    stats.register("min", np.nanmin)
    stats.register("max", np.nanmax)

    print("Start training")
    #pop = toolbox.population(n=args['POP'])
    pop_dict = {f'{ind_}':ind_ for ind_ in toolbox.population(n=args['POP']*10)}
    pop = [pop_dict[_] for _ in random.sample(list(pop_dict.keys()),args['POP'])]
    #pop += creat_pop_from_str_list(['SUE(cash_paid_for_operation_activities_mrq_0,4)'],pset)
    hof = tools.HallOfFame(args['POP'])
    info_dict = {'worlds_count': count, 
                 'EarlyStop':args['EarlyStop'],
                 'n_jobs':args['n_jobs'],
                 #'folder_path' : args['OUTPUT_TS_DATA_PATH'],
                 }
    population, logbook = algorithms.revise(
        pop, toolbox, 0.4, 0.2, n_generations=args['GEN'], n_elites=args['POP']//10, stats=stats, hall_of_fame=hof, info_dict = info_dict)

    #塞選hof
    hof_fitness_Se = pd.Series({f'{X}':X.fitness.values[0] for X in hof.items})
    new_ts_result = pd.concat({f'{ind}':ind.ts_result for ind in hof.items},axis = 1)
    #根據回傳的時序給分數
    new_df_result = Tool.CAGR(new_ts_result).sort_values(ascending = False).reset_index()
    #new_df_result = (new_ts_result.mean() / new_ts_result.std() * 252**0.5).sort_values(ascending = False).reset_index()
    #根據回傳的評分給分數
    #new_df_result = pd.Series({f'{ind}':ind.fitness.values[0] for ind in hof.items}).reset_index()
    new_df_result.columns = ['cal','fitness']
    new_df_result['use_data'] = np.nan
    new_df_result = new_df_result.loc[(new_df_result['fitness']>args['FIT_THRES']) & new_df_result['cal'].isin(hof_fitness_Se[hof_fitness_Se>0].index)]
    new_ts_result = new_ts_result[new_df_result['cal'].to_list()]

    print(f"Find {len(new_df_result)} survivors")
    print(f"update {len(set(new_df_result['cal'])-set(df_result['cal']))} survivors")
    new_df_result = new_df_result[~new_df_result['cal'].isin(df_result['cal'])]
    new_ts_result = new_ts_result[new_df_result['cal'].tolist()]

    if len(new_df_result) > 0:
        df_result = pd.concat([df_result, new_df_result], axis=0).drop_duplicates('cal', keep='first')
        df_result = df_result.sort_values('fitness', ascending=False).reset_index(drop=True)
        ts_result = pd.concat([ts_result, new_ts_result], axis=1)
        ts_result = ts_result.loc[:, ~ts_result.columns.duplicated()]

    
    # 删除高度相关的因子
    df_result,ts_result = 剔除高度相关(df_result,ts_result,Corr_threshold = args['Corr_threshold'])


    # 使用ts_result (DataFrame行是日期、列是 N 檔股票、值是日報酬)，最佳化出因子pool
    # 建立 GAStockSelection 物件
    print(f'啟動GA塞選因子池,當前有{ts_result.shape[1]}個因子')
    if ts_result.shape[1] > 1:
        ga = GAStockSelection_v4.GAStockSelection(
            ts_result=ts_result,
            pop_size=1000,         # 人口大小
            n_generations=500,   # 最多迭代500代
            mutation_rate=0.01,  # 2%突變率
            min_pick=5,
            max_pick=ts_result.shape[1],          # 最多只挑max_pick檔
            patience=50,          # 若50代都沒有更好的解就提早停
            forced_n = min(20, ts_result.shape[1]),
        )
        # 執行 fit() 進行演化
        ga.fit()
        # 取出最佳解
        solution = ga.get_best_solution()
        print("GA 找到的最佳值(演算法內估計) =", solution["best_sharpe_estimated"])
        print("實際回頭計算的 Sharpe =", solution["real_sharpe"]*252**0.5)
        print("實際回頭計算的 CAGR(%) =", solution["real_CAGR"])
        chosen_stocks = solution["chosen_stocks"].tolist()
        # 更新 df_result 與 ts_result
        df_result = df_result[df_result['cal'].isin(chosen_stocks)].sort_values('fitness', ascending=False).reset_index(drop=True)
        ts_result = ts_result[chosen_stocks]
        print(f"GA 選出 {len(chosen_stocks)} 個因子，僅保留這些因子進行後續處理")
    else:
        chosen_stocks = ts_result.columns.to_list()

    # 开始确认源资料使用频繁程度
    df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
    df_result['len'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,True))
    use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
    use_factor = use_data_df.groupby('value')['key'].apply(list)
    use_factor_count = use_factor.str.len().sort_values(ascending=False)
    使用太多次需剔除的data_name = use_factor_count.loc[use_factor_count>=args['use_data_count_threshold']].index.to_list()
    if 使用太多次需剔除的data_name:
        print(f'剔除:{使用太多次需剔除的data_name}')
    print(f'total find {len(df_result)} survivors')

    print(f'Mega Factor Score Up to:{Tool.CAGR(ts_result.mean(axis = 1))}')
    if len(df_result) < args['AMT']:
        print(f"{args['AMT'] - len(df_result)} more survivors needed")
    count += 1
    args['Operation worlds'] = f'spend {count} worlds'
    args['Operation times'] = f'Use {datetime.now()-start_time}'
    args['random seed start by'] = datetime.timestamp(start_time)
    
    save_path = args['OUTPUT_DATA_PATH']
    folder_path = os.path.dirname(save_path)
    #如果资料夹不存在直接创造资料夹
    os.makedirs(folder_path,exist_ok=True)
    ts_result_path = os.path.join(folder_path,f"{save_path.split('/')[-1].replace('.xlsx','')}.parquet")
    ts_result.to_parquet(ts_result_path)
    with pd.ExcelWriter(save_path) as writer:
        df_result.to_excel(writer, sheet_name='factor')
        pd.DataFrame.from_dict(args,orient='index').rename(columns={0:'Info'}).to_excel(writer, sheet_name='Info')
    print('='*20)
    #break

#print(df_result)
print(f'start_by:{start_time}')
print(f'end_by:{datetime.now()}')
print(f'spend {count} worlds')
print(f'Use {datetime.now()-start_time}')

Initialize Algorithm Use : 227 cols
Start training


100%|██████████| 2000/2000 [10:30<00:00,  3.17it/s]


gen	nevals	有效_mean 	avg    	std   	min   	max    
0  	2000  	0.859339	-233500	522473	-2e+06	4.67873


100%|██████████| 935/935 [02:36<00:00,  5.96it/s]


1  	935   	1.22512 	-23498.6	178740	-2e+06	4.67873


...

100%|██████████| 933/933 [02:18<00:00,  6.72it/s]


5  	933   	5.59672	-12488.2	139083	-2e+06	14.0698


100%|██████████| 938/938 [02:28<00:00,  6.32it/s]


累计1次没有变好！！！
6  	938   	3.68661	-21987.6	180323	-2e+06	14.0698


  3%|▎         | 32/967 [00:05<02:38,  5.89it/s]

In [ ]:
df_result = df_result[df_result['cal'].isin(chosen_stocks)].sort_values('fitness',ascending=False).reset_index(drop=True)
ts_result = ts_result[df_result["cal"].tolist()]

# 开始确认源资料使用频繁程度
df_result['use_data'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,False)[1])
df_result['len'] = df_result['cal'].apply(lambda X:fitness.parse_expression(X,True))
use_data_df = pd.DataFrame([(k, v) for k, values in df_result.set_index('cal')['use_data'].items() for v in values], columns=['key', 'value'])
use_factor = use_data_df.groupby('value')['key'].apply(list)
use_factor_count = use_factor.str.len().sort_values(ascending=False)
使用太多次需剔除的data_name = use_factor_count.loc[use_factor_count>=args['use_data_count_threshold']].index.to_list()
if 使用太多次需剔除的data_name:
    print(f'剔除:{使用太多次需剔除的data_name}')
print(f'total find {len(df_result)} survivors')
print(f'Mega Factor Score Up to:{Tool.CAGR(ts_result.mean(axis = 1))}')
if len(df_result) < args['AMT']:
    print(f"{args['AMT'] - len(df_result)} more survivors needed")
count += 1
args['Operation worlds'] = f'spend {count} worlds'
args['Operation times'] = f'Use {datetime.now()-start_time}'
args['random seed start by'] = datetime.timestamp(start_time)

save_path = args['OUTPUT_DATA_PATH']
folder_path = os.path.dirname(save_path)
#如果资料夹不存在直接创造资料夹
os.makedirs(folder_path,exist_ok=True)
ts_result_path = os.path.join(folder_path,f"{save_path.split('/')[-1].replace('.xlsx','')}.parquet")
ts_result.to_parquet(ts_result_path)
with pd.ExcelWriter(save_path) as writer:
    df_result.to_excel(writer, sheet_name='factor')
    pd.DataFrame.from_dict(args,orient='index').rename(columns={0:'Info'}).to_excel(writer, sheet_name='Info')
print('='*20)